In [ ]:
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import sklearn
import matplotlib

In [ ]:
######
###### Stage 1: Loading in Data 
######
######

In [ ]:
#Read in the air pollution readingS for nox emissions at the Exeter Roadside AURN air pollution monitoring stations
#The details of the station itself can be found here: 
airPollutionTargetVector = pd.read_feather("AllDataset/airPollutionTargetVector/nox/Exeter Roadside.feather")
airPollutionTargetVector["Timestamp"] = pd.to_datetime(airPollutionTargetVector["Timestamp"], utc=True)
display(airPollutionTargetVector)

In [ ]:
fig, axes = plt.subplots(1, figsize=(30,5))
axes.scatter(airPollutionTargetVector["Timestamp"], airPollutionTargetVector["nox"], s=0.01)

In [ ]:
#However as there is a considerable amount of data avaliable, this is going to be reduced to just 2018.
airPollutionTargetVector["Year"] = airPollutionTargetVector["Timestamp"].dt.year
airPollutionTargetVector_2018 = airPollutionTargetVector[airPollutionTargetVector["Year"] == 2018]
airPollutionTargetVector_2018 = airPollutionTargetVector_2018.dropna()

In [ ]:
fig, axes = plt.subplots(1, figsize=(30,5))
axes.scatter(airPollutionTargetVector_2018["Timestamp"], airPollutionTargetVector_2018["nox"], s=0.75)

In [ ]:
######
###### Stage 2: Summary Statistics of the data
######
######

airPollutionTargetVector_2018["nox"].describe()

In [ ]:
######
###### Stage 3: Find the Timestamps that are missing. 
######
######
time_range = pd.DataFrame(pd.date_range('2018-01-01T00:00:00.000Z', '2018-12-12T23:00:00.000Z', freq='H')).rename(columns={0:"Timestamp"})
airPollutionTargetVector_2018_missingTimestamps = pd.merge(airPollutionTargetVector_2018, time_range, on="Timestamp", how="right")
airPollutionTargetVector_2018_missingTimestamps = airPollutionTargetVector_2018_missingTimestamps[airPollutionTargetVector_2018_missingTimestamps['nox'].isna()]
missingTimestamps = airPollutionTargetVector_2018_missingTimestamps[["Timestamp"]]
display(missingTimestamps)

In [ ]:
#As we can see there are 297 values across the year that the monitoring stations broke down where we have no estimate for what the air pollution 
#like at the station 

#Therefore we are going to build a machine learning model that can learn a relationship and predict what the air pollution would look like in those
#timesteps.

In [ ]:
######
###### Stage 4: Model with temporal variables 
######
######

In [ ]:
#The first model that we can build could just look at the time of the day and make an estimate 

airPollutionTargetVector_2018["Hour"] = airPollutionTargetVector_2018["Timestamp"].dt.hour
airPollutionTargetVector_2018["Day"] = airPollutionTargetVector_2018["Timestamp"].dt.day
airPollutionTargetVector_2018["Week"] = airPollutionTargetVector_2018["Timestamp"].dt.isocalendar().week
airPollutionTargetVector_2018["Month"] = airPollutionTargetVector_2018["Timestamp"].dt.month
display(airPollutionTargetVector_2018)

In [ ]:
#Linear Regression model 


#In this case we are going to make a linear regression model where the target variable is the nox concentrations at a given time
y = airPollutionTargetVector_2018["nox"].to_numpy()
#The feature vector, what we are going to try and predict the pollution 
X = airPollutionTargetVector_2018[["Hour", "Day", "Week", "Month"]].to_numpy()
reg = LinearRegression().fit(X, y)


display("The score of the model is: " + str(reg.score(X, y)))
display("The coefficient is: " + str(reg.coef_) + " The intercept was: " + str(reg.intercept_))


#The model predicts that on a monday at 8AM, in the third week of a January that the air pollution would be 53.67 ug/m^3
display("Example Predicition:" + str(reg.predict(np.array([[8, 1, 3, 1]]))))

In [ ]:
#However as we can see the score for the other predicitions isnt great, we only achieve a score of 0.03
#As we know that air pollution can be affected by a range of different conditions, such as the wind speed, traffic, land use, we want to include those
#in our models 

In [ ]:
######
###### Stage 5: Read in additional data 
######
######

airPollutionFeatureVector = pd.read_feather("AllDataset/airPollutionFeatureVector/Grid_173560.feather")
airPollutionFeatureVector["Timestamp"] = pd.to_datetime(airPollutionFeatureVector["Timestamp"], utc=True)
display(airPollutionFeatureVector)

In [ ]:
#We want to combine all the data into a single dataframe to make it easier to use 
airPollutionData = pd.merge(airPollutionTargetVector_2018, airPollutionFeatureVector, on="Timestamp")
display(airPollutionData)

In [ ]:
######
###### Stage 6: Using Traffic Data 
######
######

In [ ]:
#In this case we are going to make a linear regression model where the target variable is the nox concentrations at a given time
y = airPollutionData["nox"].to_numpy()
#The feature vector, what we are going to try and predict the pollution 
X = airPollutionData[["Hour", "Day", "Week", "Month", "HGV Score", "LGV Score", "Bicycle Score", "Bus and Coach Score", "Car and Taxi Score"]].to_numpy()
reg = LinearRegression().fit(X, y)

display("The score of the model is: " + str(reg.score(X, y)))
#this time when we include additional data from the traffic aspects our performance improves to 0.2! 

In [ ]:
######
###### Stage 7: Using Met Data
######
######

In [ ]:
#What if we added in met data, such as the wind speed?

y = airPollutionData["nox"].to_numpy()
#The feature vector, what we are going to try and predict the pollution 
X = airPollutionData[["Hour", "Day", "Week", "Month", "HGV Score", "LGV Score", "Bicycle Score", "Bus and Coach Score", "Car and Taxi Score",
                     '100m_u_component_of_wind',
 '100m_v_component_of_wind',
 '10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '2m_dewpoint_temperature',
 '2m_temperature',
 'instantaneous_10m_wind_gust',
 'surface_pressure',
 'total_column_rain_water',]].to_numpy()
reg = LinearRegression().fit(X, y)

display("The score of the model is: " + str(reg.score(X, y)))

In [ ]:
#This time our performance has increased to 0.45!

######
###### Stage 8: Fill in missing data
######
######


#Lets now try and fill in the missing values that we dont have an air pollution measurement for
time_range = pd.DataFrame(pd.date_range('2018-01-01T00:00:00.000Z', '2018-12-12T23:00:00.000Z', freq='H')).rename(columns={0:"Timestamp"})
airPollutionTargetVector_2018_missingTimestamps = pd.merge(airPollutionTargetVector_2018, time_range, on="Timestamp", how="right")
airPollutionTargetVector_2018_missingTimestamps = airPollutionTargetVector_2018_missingTimestamps[airPollutionTargetVector_2018_missingTimestamps['nox'].isna()]
#airPollutionTargetVector_2018_missingTimestamps = airPollutionTargetVector_2018_missingTimestamps[airPollutionTargetVector_2018_missingTimestamps["Year"] == 2018]
missingTimestamps = airPollutionTargetVector_2018_missingTimestamps[["Timestamp"]]
display(missingTimestamps)


In [ ]:
missingTimestampsFeatureVectors = airPollutionFeatureVector[airPollutionFeatureVector["Timestamp"].isin(missingTimestamps["Timestamp"].tolist())].copy(deep=True)

In [ ]:
missingTimestampsFeatureVectors["Hour"] = missingTimestampsFeatureVectors["Timestamp"].dt.hour
missingTimestampsFeatureVectors["Day"] = missingTimestampsFeatureVectors["Timestamp"].dt.day
missingTimestampsFeatureVectors["Week"] = missingTimestampsFeatureVectors["Timestamp"].dt.isocalendar().week
missingTimestampsFeatureVectors["Month"] = missingTimestampsFeatureVectors["Timestamp"].dt.month

In [ ]:
X = missingTimestampsFeatureVectors[["Hour", "Day", "Week", "Month", "HGV Score", "LGV Score", "Bicycle Score", "Bus and Coach Score", "Car and Taxi Score",
                     '100m_u_component_of_wind',
 '100m_v_component_of_wind',
 '10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '2m_dewpoint_temperature',
 '2m_temperature',
 'instantaneous_10m_wind_gust',
 'surface_pressure',
 'total_column_rain_water',]].to_numpy()
missingAirPollutionPredicitions = reg.predict(X)
display(X.shape)
display(missingAirPollutionPredicitions.shape)
display(missingTimestamps.shape)
missingTimestamps["nox Predictions"] = missingAirPollutionPredicitions
display(missingTimestamps)

In [ ]:
fig, axes = plt.subplots(1, figsize=(30,5))
airPollutionTargetVector_2018 = airPollutionTargetVector[airPollutionTargetVector["Year"] == 2018]
axes.scatter(airPollutionTargetVector_2018["Timestamp"], airPollutionTargetVector_2018["nox"], s=0.75)
axes.scatter(missingTimestamps["Timestamp"], missingTimestamps["nox Predictions"], color="green", s=0.75)

In [ ]:
######
###### Stage 9: What are some issues with the process that has been shown so far?
######
######

In [ ]:
#Some questions you may want to think about:
#Is the data being used to calculate the score fair?
#How could we deal with the negative predictions that are beinng made?
#How could we use train and test splitting?
#When would we want to use a validation set?

In [ ]:
#However this is for data that the model has already seen, it was trained on this data.
#To get a fair representation of the model performance we want to test it on data 


######
###### Stage 10: Creating a Train, and Test Set
######
######
airPollutionDataTrainingData = airPollutionData[airPollutionData["Timestamp"] < '2018-9-1']
display(airPollutionDataTrainingData.shape)
airPollutionDataTestData = airPollutionData[airPollutionData["Timestamp"] >= '2018-9-1']
display(airPollutionDataTestData.shape)
display(airPollutionData.shape)

In [ ]:
## Your code here to use the train and test split, investigate further the true performance of the model!